# Data Journalism Lesson 9: Data Cleaning with pyjanitor

Clean up your data with code using the `pyjanitor` library.

In [ ]:
import warnings
from IPython.core.interactiveshell import InteractiveShell

# Keep hold of the real method
_orig_should_run = InteractiveShell.should_run_async

# Wrap it so that any DeprecationWarning it emits is silenced
def should_run_async(self, code, *args, **kwargs):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)
        return _orig_should_run(self, code, *args, **kwargs)

# Apply the monkey‑patch
InteractiveShell.should_run_async = should_run_async

In [ ]:
import micropip
await micropip.install('pyjanitor') 

In [ ]:
# Setup code for the notebook
import pandas as pd
import janitor  # Import pyjanitor
from IPython.display import display, HTML

# Helper functions for grading exercises
def display_feedback(correct, message_correct, message_incorrect):
    if correct:
        display(HTML(f'<div style="background-color: #dff0d8; padding: 10px; border-radius: 5px;"><strong>Correct!</strong> {message_correct}</div>'))
    else:
        display(HTML(f'<div style="background-color: #f2dede; padding: 10px; border-radius: 5px;"><strong>Not quite!</strong> {message_incorrect}</div>'))

def check_read_csv(df, expected_rows):
    correct = isinstance(df, pd.DataFrame) and len(df) == expected_rows
    msg_correct = 'Great work! You imported your state\'s census of governments.'
    msg_incorrect = 'Check the file path or the read_csv function. Did you replace ____ with your state?'
    if not isinstance(df, pd.DataFrame):
        msg_incorrect = 'The variable `governments` should be a pandas DataFrame.'
    elif len(df) != expected_rows:
         msg_incorrect = f'Expected {expected_rows} rows, but got {len(df)}. Did you load the correct state\'s data?'
    display_feedback(correct, msg_correct, msg_incorrect)

def check_glimpse(df_info_output):
    # info() prints to stdout and returns None.
    # We assume if the code runs and info_output is None, the user called it.
    correct = df_info_output is None
    msg_correct = 'Good job using .info() to inspect the DataFrame!'
    msg_incorrect = 'Make sure you are calling the .info() method on the `governments` DataFrame.'
    display_feedback(correct, msg_correct, msg_incorrect)

def check_clean_names(df):
    correct = False
    msg_correct = 'Excellent! The column names are now cleaned (snake_case).'
    msg_incorrect = 'Make sure you applied the .clean_names() method correctly.'
    if isinstance(df, pd.DataFrame):
        # Check if names look like snake_case (simple check)
        are_snake_case = all([' ' not in col and col.lower() == col for col in df.columns])
        if are_snake_case:
             correct = True
        else:
             msg_incorrect = 'The column names do not appear to be in snake_case. Did you use .clean_names()?'
    else:
        msg_incorrect = 'Expected a DataFrame as output.'
    display_feedback(correct, msg_correct, msg_incorrect)

def check_remove_empty(df, original_df):
    correct = isinstance(df, pd.DataFrame) # Basic check
    msg_correct = 'Good! You\'ve applied the remove_empty() method.'
    msg_incorrect = 'Make sure you are calling .remove_empty() on the cleaned DataFrame.'
    # More robust check could compare shapes if empty rows/cols were expected
    display_feedback(correct, msg_correct, msg_incorrect)

def check_tabyl(result_df, expected_col):
    correct = False
    msg_correct = f'Successfully generated the frequency table for {expected_col}!'
    msg_incorrect = f'Could not verify the frequency table for {expected_col}.'
    if isinstance(result_df, pd.DataFrame):
        # janitor.tabyl column names can vary slightly based on version/usage
        count_col_present = 'n' in result_df.columns or 'count' in result_df.columns
        percent_col_present = 'percent' in result_df.columns or 'percentage' in result_df.columns
        if expected_col in result_df.columns and count_col_present and percent_col_present:
            correct = True
        else:
            msg_incorrect = f'Expected columns like \'{expected_col}\', a count column (e.g., \'n\', \'count\'), and a percent column (e.g., \'percent\') in the output DataFrame. Did you use janitor.tabyl correctly?'
    else:
        msg_incorrect = 'Expected a DataFrame output from janitor.tabyl().'
    display_feedback(correct, msg_correct, msg_incorrect)

In [ ]:
# --- Load Data Setup ---
state_full_name = "Minnesota"
state_lower_url = state_full_name.lower().replace(' ', '-') # Handle spaces for URL
dataurl = f'../_static/government-census/{state_lower_url}.csv'

# Load the main dataset
governments = pd.read_csv(dataurl)
nrows = len(governments)
ncolumns = len(governments.columns)

countyLanguage = ('county', 'counties')

# Pre-clean data for later exercises
clean_governments = governments.clean_names().remove_empty() # Default removes both rows and cols

In [ ]:
# Glue variables for use in markdown
from myst_nb import glue

glue("state_full_name", state_full_name, display=False)
glue("nrows", nrows, display=False)
glue("ncolumns", ncolumns, display=False)
glue("county_singular", countyLanguage[0], display=False)
glue("county_plural", countyLanguage[1], display=False)

## The Goal

In this lesson, you'll learn how to use the `pyjanitor` package to clean and standardize your data more efficiently. By the end of this tutorial, you'll understand how to clean column names, remove empty rows and columns, identify duplicates, and explore data consistency using `pyjanitor` functions. These skills will help you prepare messy datasets for analysis more quickly and reliably, saving you time and reducing errors in your data journalism work.

## What is Data Journalism?

`pyjanitor` is one of those Python packages that honestly, I never knew I needed until I *really* needed it. Now it's one of the packages I use the most.

What does `pyjanitor` do? Think of it like a Swiss Army Knife of data cleaning for pandas DataFrames. It's got a knife, some scissors, a file, a bottle opener, and on and on. Do they all go together? It doesn't seem like it until you're camping and having all of those things in a small package is really handy.

Enter `pyjanitor`. Someday, you'll get a dataset with 300 *columns* of data and most of them have names that don't conform to Python's conventions (or just good practice). Names like "1st Quarter 2024 Actual" or "2024 Estimated Revenue". Spaces, numbers starting columns, symbols -- a horrorshow. And `pandas` will dutifully read that, but using those column names in your code is a pain (requiring `df['Column Name With Spaces']` syntax).

What if you could fix them all at once? You can with `pyjanitor`. It has a `clean_names()` method that does this.

Did you get some funky spreadsheet from a government entity with lots of blank rows and columns? `pyjanitor` can remove columns and rows with no data in them automatically using `remove_empty()`.

Using Python to try and read in hundreds of spreadsheets programmatically, but those spreadsheets put the header row on line 6 instead of line 1 like you're supposed to? `pyjanitor` can handle that with `skip_header()` (though `pandas.read_csv` also has a `header` argument).

And a lot more. Let's explore.

## The Basics

The bane of every data analyst's existence is data cleaning.

Every developer, every data system, every agency, they all have opinions about how data gets collected. Some decisions make sense from the outside. Some decisions are based entirely on internal politics: who is creating the data, how they are creating it, why they are creating it. Is it automated? Is it manual? Are data normalized -- meaning all the spellings are the same? Are there free form fields where users can just type into or does the system restrict them to choices?

Your question -- what you want to do with the data -- is almost never part of that equation.

So cleaning data is the process of fixing issues in your data so you can answer the questions you want to answer. Unfortunately, there's no template here. There's no checklist. It's just a big bag of tricks that eventually runs out and you'll be left fixing individual issues by hand, if it's really bad.

But let's start simple. There are certain tricks that we can start with to make our lives easier. We'll slowly make it harder as we dig deeper.

One of the first places we can clean data is cleaning the headers. Every system has their own way of recording headers, and every developer has their own thoughts of what a good idea is within it. Python (and pandas) is most happy when headers are one word (using underscores instead of spaces), lower case, without special characters (except underscores). This is often called "snake_case". Without telling you, I've been fixing these headers behind the scenes before you get the data. No more. Today, you're going to notice that pandas will read columns that have spaces or start with numbers or have special characters in them, but accessing them requires bracket notation (e.g., `df['Column Name']`) instead of dot notation (e.g., `df.column_name`).

There is an external library in Python called `pyjanitor` that makes fixing headers trivially simple.

First things first, as always: we load the libraries we need.

In [ ]:
import pandas as pd
import janitor

Now we load our data. I'm writing this chapter during the last few months of the 2024 Presidential Election. A common task for journalists during elections? Fact-checking. A common trope in elections are claims that government has become too big, too bloated, too expensive. Your first thought, as a reporter, should be simply: Has it? A way to check? The U.S. Census Bureau. The same agency who counts people and estimate populations between counts do a ton of other enumerations in and around American society. One is the annual survey of governments, where the Census Bureau asks every government -- state, {glue:text}`county_singular`, city, school district -- how many people they employ and what their payroll is. It's a fantastic dataset for comparing your community to others like it, in your state and in the nation. Got a mayoral candidate claiming the city has tripled the number of people working there and is out of control? You can check that.

We get this data the same way we have all along:

In [ ]:
governments = pd.read_csv("../_static/government-census/minnesota.csv")
governments.head()

## Cleaning headers

You can get a sense of the problems with the headers in this data with a quick `.info()`:

In [ ]:
governments._____()

Look at the column names listed. You'll likely see names with spaces and mixed capitalization like `Name of Government` or `Type of Government`. If you wanted to use this column in a filter, it would look like this: `governments[governments['Type of Government'] == 'State']`. That's a lot of extra things to go wrong -- did you capitalize correctly, did you remember the quotes around the column name? It would be better if we could simplify this and reduce the number of places where we could go wrong by converting to snake_case (`type_of_government`).

`pyjanitor` makes this easy to fix. How easy? This easy. `pyjanitor` provides a `.clean_names()` method that you can chain onto your DataFrame.

### Exercise 1: Cleaning up column names

In [ ]:
cleaned_df_names = governments._____()
cleaned_df_names.info()

Just like that, all lowercase, all one word (using underscores), no awkward bracket notation necessary to confuse our code later on.

### Exercise 2: Dropping empty columns and rows

Another good trick by `pyjanitor` is easily dropping empty columns and rows. Sometimes columns are in the data and there's nothing in them. Nada. Blank. Rarer, but still possible: Rows of blank data. We could use pandas' `.dropna()` but `pyjanitor`'s `remove_empty()` provides a convenient wrapper specifically for completely empty rows/columns.

Because this is data from the Census Bureau, an agency who exists to create data, we have a minimum of weirdness to deal with. But a good plan when you begin working with data is to inspect the data after you import it, and then apply `pyjanitor` functions to clean it up. `pyjanitor` has a function called `.remove_empty()` which by default removes both empty rows and columns. You can specify `axis='rows'` or `axis='columns'` if needed.

This likely won't remove anything from this specific dataset, but this is how you would do it:

In [ ]:
cleaned_and_emptied_df = governments.clean_names()._____()
cleaned_and_emptied_df.head()

Let's save what we've done so we can use it for the rest of the exercise into a new dataframe called `clean_governments`.

In [ ]:
# Save the cleaned data (re-assigning based on previous step)
clean_governments = cleaned_and_emptied_df

# Verify by checking columns
print(clean_governments.columns)

## Inconsistency

`pandas` itself also has some handy tools for our data smells. One is called `value_counts`, which creates a frequency table of unique records in a single field. All you need to do is call `dataframe.value_counts('column_name')` with the DataFrame and column name.

### Exercise 3: Data smells with `value_counts`

So does the Census Bureau record the names of governments consistently? `value_counts` will tell us and will tell us a little bit about the data. Remember: we've cleaned the headers, so it's now `name_of_government`.

In [ ]:
name_counts = clean_governments._____(_____)
name_counts

What you are looking for here are mistakes. For a made up example: Let's pretend you had a town named Adams. Do you see `adams` and then `ADAMS` (after cleaning)? There's a chance that's a mistake (though `clean_names` helps with case). More likely, you might see variations like `adams county` and `adams city`. There's a chance you might have two different levels of government called that. Data smells are just a warning -- they aren't a guarantee of problems.

### Exercise 4: Combining what we know

What if you had a legislative candidate complaining about how expensive running the jails had become? We can add a pandas `filter` (using boolean indexing) before we run `value_counts` to give specific data the smell test. To find jails (and prisons), we want to filter the `government_function` column for 'Corrections'. Then, we can run `value_counts` on the `name_of_government` column for that filtered data to see what we get.

In [ ]:
corrections_df = clean_governments[clean_governments['government_function'] == _____]
corrections_counts = corrections_df._____(_____)
print(corrections_counts)

Depending on how your state is set up, you might see one row for your state corrections organization -- probably under your state's name -- and one row for each {glue:text}`county_singular` jail. If you see counts with a number other than 1, that's an immediate question you'll need to answer before you start doing data analysis. Why are there multiple entries for the same government name within the 'Corrections' function?

## The Recap

Throughout this lesson, you've learned several key data cleaning techniques using the `pyjanitor` package with pandas. You've practiced cleaning column names with `.clean_names()`, removing empty rows and columns with `.remove_empty()`, and exploring data consistency with `value_counts`. Remember, data cleaning is an essential step in any data analysis project, and the tools you've learned here will help you tackle common data issues more efficiently. As you work with different datasets, you'll find these `pyjanitor` functions invaluable for quickly standardizing and exploring your data before diving into deeper analysis.

## Terms to Know

- **`pyjanitor`**: A Python package providing convenient data cleaning routines for pandas DataFrames.
- **`.clean_names()`**: A `pyjanitor` method that cleans column names, typically converting them to snake_case.
- **`.remove_empty()`**: A `pyjanitor` method that removes empty rows and/or columns from a DataFrame.
- **Data cleaning**: The process of identifying and correcting errors or inconsistencies in datasets.
- **Duplicates**: Multiple identical records in a dataset that may skew analysis results (can be found with pandas' `.duplicated()` and `.drop_duplicates()`).
- **Data consistency**: The uniformity and reliability of data across a dataset.
- **snake_case**: A naming convention where words are lowercase and separated by underscores (e.g., `column_name`).